# Agenda

1. Functions
    - Inner functions
    - Dispatch tables
    - Bytecodes and compilation
2. Mypy + annotations
    - Type annotations / type hints
    - Mypy 
    - Modern techniques
3. Objects
    - What happens when we create an object?
    - Attributes (ICPO)
    - Methods 
    - Magic methods
    - Inheritance

# Inner functions

1. Functions are objects (can be arguments to functions, can be return values from functions)
2. When we use `def`, we (a) create a function object and (b) assign to a variable
3. When we assign to a variable in a function, the variable is local

In [1]:
def outer():
    def inner():
        print('I am in inner!')
    return inner

x = outer()    

In [2]:
type(x)

function

In [3]:
x.__name__

'inner'

In [4]:
x()

I am in inner!


In [5]:
def hello():
    return 'Hello!'

In [6]:
hello.__name__

'hello'

In [7]:
y = hello

In [8]:
y.__name__

'hello'

In [9]:
del(hello)

In [10]:
y()

'Hello!'

In [ ]:
def outer(a):
    def inner(b):
        print(f'I am in innerm {a=}, {b=}!')
    return inner

x = outer()    